## Expert Knowledge Worker

### A question answering agent that is an expert knowledge worker
### To be used by employees of Insurellm, an Insurance Tech company
### The agent needs to be accurate and the solution should be low cost.

This project will use RAG (Retrieval Augmented Generation) to ensure our question/answering assistant has high accuracy.

This first implementation will use a simple, brute-force type of RAG..

In [1]:
!pip install requests beautifulsoup4 --quiet

In [2]:
import requests
from bs4 import BeautifulSoup

def get_page_text(url):
    """Verilen URL'den temizlenmiş metin içerik döner."""
    response = requests.get(url)
    if response.status_code != 200:
        print(f"❌ Failed to fetch {url}")
        return ""
    
    soup = BeautifulSoup(response.content, 'html.parser')
    
    # Header, footer, script gibi içerikleri çıkar
    for tag in soup(['script', 'style', 'nav', 'footer']):
        tag.decompose()

    text = soup.get_text(separator='\n', strip=True)
    return text


In [5]:
pages = {
    "home": "https://www.eliteconstructionde.com/",
    "about": "https://www.eliteconstructionde.com/about",
    "contact": "https://www.eliteconstructionde.com/contact",
    "bathroom": "https://www.eliteconstructionde.com/bathroom-remodeling-services/",
    "kitchen": "https://www.eliteconstructionde.com/kitchen-remodeling-services/",
    "basement": "https://www.eliteconstructionde.com/basement-contractors/",
    "room_addition": "https://www.eliteconstructionde.com/room-addition-contractors/",
    "deck": "https://www.eliteconstructionde.com/deck-services/",
    "repair": "https://www.eliteconstructionde.com/residential-repair-services/"
}

raw_documents = []
for name, url in pages.items():
    text = get_page_text(url)
    print(f"✅ Collected {name} page, {len(text)} characters")
    raw_documents.append({
        "page": name,
        "url": url,
        "text": text
    })


✅ Collected home page, 7055 characters
✅ Collected about page, 2556 characters
✅ Collected contact page, 712 characters
✅ Collected bathroom page, 6172 characters
✅ Collected kitchen page, 6560 characters
✅ Collected basement page, 3497 characters
✅ Collected room_addition page, 3659 characters
✅ Collected deck page, 6266 characters
✅ Collected repair page, 5671 characters


In [6]:
!pip install langchain openai tiktoken chromadb --quiet


In [7]:
from langchain.schema import Document
from langchain.text_splitter import CharacterTextSplitter


In [8]:
documents = []

for doc in raw_documents:
    metadata = {
        "source": doc["url"],
        "page": doc["page"]
    }
    documents.append(Document(page_content=doc["text"], metadata=metadata))


In [9]:
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
chunks = text_splitter.split_documents(documents)

print(f"✅ Total chunks: {len(chunks)}")
print(f"📄 Chunk örneği:\n\n{chunks[0].page_content[:500]}...")
print(f"🔖 Metadata: {chunks[0].metadata}")


✅ Total chunks: 9
📄 Chunk örneği:

Home Remodeling Contractors Georgetown, DE | Elite Construction
Plan Your Remodel Today!
Walls Out, Worries Gone
4.70/5.00
Over 24 Happy Customers!
Let Me Know What You Need Done!
Have a home project in mind? I would love to help. Please fill out the contact form below, and I will get back to you as soon as I can.
first name
last name
email
Assistive text
phone number
Assistive text
message
Δ
Leave this field empty
Crafting Comfortable Homes From Within
Your Dependable
Home Remodeling Contractor...
🔖 Metadata: {'source': 'https://www.eliteconstructionde.com/', 'page': 'home'}


In [10]:
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma
import os
from dotenv import load_dotenv

load_dotenv()


True

In [11]:
embedding = OpenAIEmbeddings()


C:\Users\Onur\AppData\Local\Temp\ipykernel_45884\3993146642.py:1: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embedding = OpenAIEmbeddings()


In [12]:
db_name = "elite_vector_db"

# Önceki varsa sil (isteğe bağlı)
import shutil
if os.path.exists(db_name):
    shutil.rmtree(db_name)


In [13]:
vectorstore = Chroma.from_documents(
    documents=chunks,
    embedding=embedding,
    persist_directory=db_name
)

print(f"✅ Vector DB created with {vectorstore._collection.count()} vectors.")


✅ Vector DB created with 9 vectors.


In [14]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory


In [15]:
llm = ChatOpenAI(model_name="gpt-4o", temperature=0.3)


C:\Users\Onur\AppData\Local\Temp\ipykernel_45884\3059087854.py:1: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(model_name="gpt-4o", temperature=0.3)


In [16]:
retriever = vectorstore.as_retriever()

memory = ConversationBufferMemory(
    memory_key="chat_history",
    return_messages=True
)


C:\Users\Onur\AppData\Local\Temp\ipykernel_45884\1056145233.py:3: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(


In [24]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory

llm = ChatOpenAI(model_name="gpt-4o", temperature=0.3)

retriever = vectorstore.as_retriever()

memory = ConversationBufferMemory(
    memory_key="chat_history",
    return_messages=True
)

conversation_chain = ConversationalRetrievalChain.from_llm(
    llm=llm,
    retriever=retriever,
    memory=memory
)


In [25]:
query = "What services does Elite Construction offer?"
result = conversation_chain.invoke({"question": query})
print(result["answer"])


Elite Construction offers a range of services, including:

1. Kitchen Remodeling: Custom kitchen remodeling services to improve layout, storage, and style.
2. Bathroom Remodeling: Transforming bathrooms with a focus on comfort, functionality, and style.
3. Basement Remodeling: Converting unfinished basements into comfortable, usable spaces.
4. Room Addition: Adding space to homes, whether building out or up.
5. General Residential Repairs: Addressing everyday issues quickly with reliable repair services.
6. Deck Services: Custom deck design, repair, and installation for outdoor living spaces.
7. General Residential Repairs: Fast, reliable repairs for issues like leaky faucets, cracked drywall, and more.

These services are tailored to meet the specific needs of homeowners in Georgetown, DE, and nearby areas.


In [26]:
import gradio as gr


In [27]:
def chat(message, history):
    result = conversation_chain.invoke({"question": message})
    return result["answer"]


In [28]:
interface = gr.ChatInterface(
    fn=chat,
    type='messages',
    title="🏠 Elite Construction Assistant",
    description="Ask anything about Elite Construction's services, contact, or company background.",
    theme="soft"
)

interface.launch(share=True)


* Running on local URL:  http://127.0.0.1:7877
* Running on public URL: https://0f781f97ec1b3a0639.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
